Where does the *avg* function find the series?

Note that **series** is a local variable of *make_averager* because the assignment **series** = [] happens in the body of that function.

But when *avg(10)* is calledm, *make_averager* has already returned, and its local scope is long gone.

Within *averager*, **series** is a *free variable*.

![closure](free_)var.png

*The closure of the average extends the scope of that function to include the binding for the free variable series.*

In [3]:
def make_averager():
    series = []                     ## LOCAL VARIABLE FOR MAKE_AVERAGER

    def averager(new_value):
        series.append(new_value)    ## SERIES IS A FREE VARIABLE
        total = sum(series)

        return total / len(series)
    
    return averager

avg = make_averager()

print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [2]:
#### closure II ####

series = []                       ## GLOBAL VARIABLE

def make_averager(): 
    # series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        
        return total / len(series)
    
    return averager

avg = make_averager()

print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
()


The __code__ attribute of a function represents the compiled body of the function

The body of a function is evaluated in the environment where the function is defined, not the environment where the func is called.

In [22]:
######## non local ########

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        # count = 1             # error because local need 'nonlocal' # Or use 'global' 
        nonlocal count, total
        count += +1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(1))


10.0
5.5


In [24]:
import time

def snooze(s):
    time.sleep(s)
    print('bla')
    return 'wake up'

snooze(5)

bla


In [45]:
import time

def snooze(s):
    print(f'{s} seconds to sleep')
    time.sleep(s)
    print('bla')
    return 'wake up'

def clock(fun):
    def clocked(n):
        start_time = time.time()
        print('TIME START', start_time)

        result = fun(n)

        end_time = time.time()

        elapsed = end_time - start_time
        print(f'It took {elapsed}s.')


    return clocked


snooze1 = clock(snooze)

snooze1(2)

snooze = clock(snooze)

TIME START 1685015424.6574364
2 seconds to sleep
bla
It took 2.0005409717559814s.


In [43]:
@clock
def snooze(s):
    print(s)
    time.sleep(s)
    print('bla')
    return 'Wake up'

snooze(0.5)

@clock
def test(s):
    print(s)
    time.sleep(s)
    print('test')
    return 'wake up'

test(1)


TIME START 1685015364.3129835
0.5
bla
It took 0.50113844871521s.
TIME START 1685015364.814122
1
test
It took 1.000375509262085s.


'wake up BOB'

In [55]:
@clock
def fac(n):   # 1*
    result = 1
    while n >= 1:
        result = result * n
        n -= 1
    return result

print(fac(10))

TIME START 1685015492.8028848
It took 0.0s.
None


In [84]:
def decorator(func):
    def inner(*args, **kwargs):
        return 'deco works ' + func(*args, **kwargs)
    return inner

@decorator
def test(word1, word2 = 'WORLD'):
    return f'{word1} {word2}'

test('HELLO')    

'deco works HELLO WORLD'

In [87]:
def make_bold(func):
    def inner():
        return f'<strong> {func()} </strong>'
    return inner

def get_html_greetings():
    return f'Hello World'

get_html_greetings = make_bold(get_html_greetings) # Type Function
print(get_html_greetings())

<strong> Hello World </strong>


In [91]:
def wrap_with(tag='div'):
    def decorator(func):
        def inner():
            return f'<{tag}>{func()}</{tag}>'
        return inner
    return decorator

def test(w1, w2='world'):
    return f'{w1} {w2}'
test = wrap_with(tag='strong')(get_html_greetings)
print(test())

<strong><strong> Hello World </strong></strong>


In [95]:
@wrap_with('strong')
@wrap_with('p')
def get_html_greetings():
    return f'Hello world'

print(get_html_greetings())

<strong><p>Hello world</p></strong>
